In [95]:
import pandas as pd
import numpy as np

VarMatrixForm = pd.read_excel(r'Source.xlsx', sheet_name='VarMatrix') # Получаем форму матрицы переменных
ProfitabilityMatrix = pd.read_excel(r'Source.xlsx', sheet_name='Profit_decay') # Получаем параметры прибыльности по каждой единице товара
VolumeMatrix = pd.read_excel(r'Source.xlsx', sheet_name='Volume') # Получаем данные об объеме каждой единицы товара
Boxes_param = pd.read_excel(r'Source.xlsx', sheet_name='Boxes_param') # Получаем данные о возможной вместимости коробок и цене отправки

np_Box = Boxes_param['Box']
np_ProfitabilityMatrix = np.array(ProfitabilityMatrix)
np_VolumeMatrix = np.array(VolumeMatrix)
np_Boxes_param = np.array(Boxes_param)

import pulp as pl
model = pl.LpProblem("Send_plan", pl.LpMaximize)

variables = pl.LpVariable.matrix('LPmodel', (np_Box, VarMatrixForm.keys().values), 0, 1, cat='Integer') # формируем матрицу переменных (строки - коробки, столбцы - товары, первый столбец в каждой строке - переменная-флаг), область допустимых значений - 0 или 1
VarMatrix = np.array(variables).reshape((VarMatrixForm.shape[0]), (VarMatrixForm.shape[1])) # Преобразуем в форму Boxes * Products

objective = pl.lpSum(VarMatrix[:,1:] * np_ProfitabilityMatrix) - VarMatrix[:,:1] * np_Boxes_param[:,1].reshape(2,1) # целевая функция

model += objective

# limit volume for boxes
for i in range(int(np_Boxes_param.shape[0])):
    model += pl.lpSum(VarMatrix[i, 1:] * np_VolumeMatrix) <= np_Boxes_param[i][2] # Сумма объемов товаров в коробке должна быть не больше вместимости коробки

# we can send one product only
for j in range(1, VarMatrix.shape[1]): # почему "бежим" по колонкам, начиная с 1, а не с 0 - т.к. в столбце 0 у нас хранятся переменные - флаги для коробок
    model += pl.lpSum(VarMatrix[:,j]) <= 1.0 # нельзя один товар положить сразу в две коробки

# flag var
for i in range(VarMatrix.shape[0]):
    for j in range(1, VarMatrix.shape[1]):
        model += VarMatrix[i][j] <= VarMatrix[i][0] 

       
model.solve(pl.PULP_CBC_CMD( msg = True))

Solution = pd.DataFrame()
if any((v.varValue or 0) < 0 for v in model.variables()) is False:
    for v in model.variables():
        if not v.varValue:
            continue
        name = v.name.replace('LPmodel_', '').split('_')
        qty = v.varValue
        print(f'{name}: {qty}')
            

['Box2', 'Product1', '1']: 1.0
['Box2', 'Product1', '2']: 1.0
['Box2', 'Product1', '3']: 1.0
['Box2', 'Product2', '1']: 1.0
['Box2', 'Product2', '2']: 1.0
['Box2', 'Product2', '3']: 1.0
['Box2', 'Product2', '4']: 1.0
['Box2', 'Product4', '1']: 1.0
['Box2', 'Product4', '2']: 1.0
['Box2', 'Product5', '1']: 1.0
['Box2', 'Product5', '2']: 1.0
['Box2', 'Product5', '3']: 1.0
['Box2', 'Product5', '4']: 1.0
['Box2', 'box', 'flag']: 1.0
